# Does size matter in MMA?

In [298]:
import pandas as pd
import scipy.stats as stats

fights = pd.read_csv('fights_overview.csv')
fighters = pd.read_csv('fighters.csv')

fights = pd.merge(fights, fighters, left_on="fighter_1_link", right_on="fighter_link")
fights = fights[["fighter_1_link", "height", "reach", "fighter_1_result", "fighter_2_link", "fighter_2_result"]]
fights.rename(columns={'height': 'fighter_1_height', 'reach': "fighter_1_reach"}, inplace=True)
fights = pd.merge(fights, fighters, left_on="fighter_2_link", right_on="fighter_link")
fights = fights[["fighter_1_link", "fighter_1_height", "fighter_1_reach", "fighter_1_result", 
                 "fighter_2_link", "height", "reach" ,"fighter_2_result"]]
fights.rename(columns={'height': 'fighter_2_height', 'reach': "fighter_2_reach"}, inplace=True)
fights = fights.dropna()
fights = fights[~fights['fighter_1_result'].isin(['NC', 'D'])]

def find_height_diff(row):
    if row['fighter_1_result'] == "L":
        return row['fighter_2_height'] - row['fighter_1_height']  
    else:
        return row['fighter_1_height'] - row['fighter_2_height']  

def find_reach_diff(row):
    if row['fighter_1_result'] == "L":
        return row['fighter_2_reach'] - row['fighter_1_reach']  
    else:
        return row['fighter_1_reach'] - row['fighter_2_reach']  

## The height difference column is define by winner's height - loser's height
## The reach difference column is define by winner's reach - loser's reach

fights["height_diff"] = fights.apply(find_height_diff, axis=1)
fights["reach_diff"] = fights.apply(find_reach_diff, axis=1)

## First hypothesis test
H<sub>0</sub>= mean(height_diff) = 0

H<sub>A</sub>= mean(height_diff) &ne; 0

In [301]:
p_value = (stats.ttest_1samp(fights['height_diff'], 0))[1]
print("p-value:", p_value)

p-value: 0.000613892673543839


## Second hypothesis test
H<sub>0</sub>= mean(height_reach) = 0

H<sub>A</sub>= mean(height_reach) &ne; 0

In [304]:
p_value = (stats.ttest_1samp(fights['reach_diff'], 0))[1]
print("p-value:", p_value_2)

p-value: 4.071697939923229e-07


In [306]:
print(fights['height_diff'].mean())
print(fights['reach_diff'].mean())

0.103770783847981
0.2002672209026128


# Conclusion

Both null hypotheses are rejected meaning there is evidence that both height and reach influence the result of the fight. Winners are on average 0.1 inches (0.25 cm) taller than their opponent have have a reach 0.2 inches (0.5 cm) longer than their opponent.